# Motor Hat STM32 & Peripherals

[STM32F405 Pinout](../figures/feather-pinout.pdf "Pinout")

* Adafruit "Recommended" pin functions don't match needs, use different assignments
* Consult Table 9 in STM32F405 manual for pin "alternate functions"

In [1]:
%register 'serial:///dev/serial0'
%connect robot-stm

Connected to robot-stm @ serial:///dev/serial0


## Pin Functions

In [1]:
# pin alternate functions

from pyb import Pin

print("{:15} {:4} {}".format('board name', 'name', 'alternate functions'))
for p in sorted(dir(Pin.board)):
    if '__' in p: continue
    pin = Pin(p)
    names = pin.names()
    print("{:15} {:4} {}".format(names[1], names[0], pin.af_list()))

board name      name alternate functions
AIN1            C5   []
AIN2            C3   [Pin.AF5_SPI2]
AUX             C4   []
BATTERY_MONITOR A3   [Pin.AF1_TIM2, Pin.AF2_TIM5, Pin.AF3_TIM9, Pin.AF7_USART2]
BIN1            C2   [Pin.AF5_SPI2]
BIN2            A4   [Pin.AF5_SPI1, Pin.AF7_USART2]
BOOT1           B2   []
CTS             B13  [Pin.AF1_TIM1, Pin.AF5_SPI2, Pin.AF7_USART3]
D8              C0   []
ENC_A1          A6   [Pin.AF1_TIM1, Pin.AF2_TIM3, Pin.AF3_TIM8, Pin.AF5_SPI1, Pin.AF9_TIM13]
ENC_A2          A7   [Pin.AF1_TIM1, Pin.AF2_TIM3, Pin.AF3_TIM8, Pin.AF5_SPI1, Pin.AF9_TIM14]
ENC_B1          B6   [Pin.AF2_TIM4, Pin.AF4_I2C1]
ENC_B2          B7   [Pin.AF2_TIM4, Pin.AF4_I2C1]
FLASH_CS        A15  [Pin.AF1_TIM2, Pin.AF1_TIM2, Pin.AF5_SPI1]
FLASH_MISO      B4   [Pin.AF2_TIM3, Pin.AF5_SPI1]
FLASH_MOSI      B5   [Pin.AF2_TIM3, Pin.AF5_SPI1]
FLASH_SCK       B3   [Pin.AF1_TIM2, Pin.AF5_SPI1]
NC_A0           A0   [Pin.AF1_TIM2, Pin.AF1_TIM2, Pin.AF2_TIM5, Pin.AF3_TIM8, Pin.AF7_USART2]

## PWM Motor Control

![TB6612 breakout](../figures/TB6612-breakout.jpg "breakout")

* Channel A control: PWMA, AIN1, AIN2
* Channel B control: PWMB, BIN1, BIN2

| Function | IN1 | IN2 | Notes |
|----------|-----|-----|-------|
| Forward  |  L  |  H  |  
| Reverse  |  H  |  L  |
| Brake    |  H  |  H  | speed=0 has same effect
| Coast (Z)|  L  |  L  |


In [1]:
from tb6612 import TB6612
from pyb import Pin, Timer

freq = 10_0000
tim = Timer(8, freq=freq)

motor = TB6612(
    [ tim.channel(3, Timer.PWM_INVERTED, pin=Pin(Pin.cpu.B15)),
      Pin(Pin.cpu.C4,  mode=Pin.OUT_PP),
      Pin(Pin.cpu.A5,  mode=Pin.OUT_PP) ],
    [ tim.channel(2, Timer.PWM_INVERTED, pin=Pin(Pin.cpu.B14)),
      Pin(Pin.cpu.C5,  mode=Pin.OUT_PP),
      Pin(Pin.cpu.B13, mode=Pin.OUT_PP) ]
)

PWM started


In [1]:
motor.speed_a(0)
motor.speed_b(90)

## Encoder

* Timers 2-5 can handle quadrature inputs
* Tim2 and Tim5 are 32-bit: Preferable! (but not available on Adafruit Feather 405)
* Use Tim3 & Tim4 (16-bit)

![Encoders](../figures/encoder.jpg "encoder")

In [1]:
import time
from encoder import init_encoder, c2

enc1 = init_encoder(3, Pin.cpu.A6, Pin.cpu.A7, Pin.AF2_TIM3)
enc2 = init_encoder(4, Pin.cpu.B6, Pin.cpu.B7, Pin.AF2_TIM4)

In [ ]:
for i in range(50):
    print("1", c2(enc1.counter()))
    enc1.counter(0)
    time.sleep(1)

## IMU

In [1]:
%connect robot-stm

from machine import I2C as I2C
from pyb import Pin
from bno055 import BNO055


Connected to robot-stm @ serial:///dev/serial0
